In [1]:
import gym
import numpy as np

In [2]:
env = gym.make("Pong-v0")
observation = env.reset() # This gets us the image

# hyperparameters
episode_number = 0
batch_size = 10
gamma = 0.99 # discount factor for reward
decay_rate = 0.99
num_hidden_layer_neurons = 200
input_dimensions = 80 * 80
learning_rate = 1e-4

episode_number = 0
reward_sum = 0
running_reward = None
prev_processed_observations = None

weights = {
    '1': np.random.randn(num_hidden_layer_neurons, input_dimensions) / np.sqrt(input_dimensions),
    '2': np.random.randn(num_hidden_layer_neurons) / np.sqrt(num_hidden_layer_neurons)
}

# To be used with rmsprop algorithm (http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop)
expectation_g_squared = {}
g_dict = {}
for layer_name in weights.keys():
    expectation_g_squared[layer_name] = np.zeros_like(weights[layer_name])
    g_dict[layer_name] = np.zeros_like(weights[layer_name])

episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [], [], [], []

In [28]:
episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [], [], [], []

def downsample(image):
    # Take only alternate pixels - basically halves the resolution of the image (which is fine for us)
    return image[::2, ::2, :]

def remove_color(image):
    """Convert all color (RGB is the third dimension in the image)"""
    return image[:, :, 0]

def remove_background(image):
    image[image == 144] = 0
    image[image == 109] = 0
    return image

def preprocess_observations(input_observation, prev_processed_observation, input_dimensions):
    """ convert the 210x160x3 uint8 frame into a 6400 float vector """
    processed_observation = input_observation[35:195] # crop
    processed_observation = downsample(processed_observation)
    processed_observation = remove_color(processed_observation)
    processed_observation = remove_background(processed_observation)
    processed_observation[processed_observation != 0] = 1 # everything else (paddles, ball) just set to 1
    # Convert from 80 x 80 matrix to 1600 x 1 matrix
    processed_observation = processed_observation.astype(np.float).ravel()

    # subtract the previous frame from the current one so we are only processing on changes in the game
    if prev_processed_observation is not None:
        input_observation = processed_observation - prev_processed_observation
    else:
        input_observation = np.zeros(input_dimensions)
    # store the previous frame so we can subtract from it next time
    prev_processed_observations = processed_observation
    return input_observation, prev_processed_observations


def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def relu(vector):
    vector[vector < 0] = 0
    return vector

def apply_neural_nets(observation_matrix, weights):
    """ Based on the observation_matrix and weights, compute the new hidden layer values and the new output layer values"""
    hidden_layer_values = np.dot(weights['1'], observation_matrix)
    hidden_layer_values = relu(hidden_layer_values)
    output_layer_values = np.dot(hidden_layer_values, weights['2'])
    output_layer_values = sigmoid(output_layer_values)
    return hidden_layer_values, output_layer_values

def choose_action(probability):
    random_value = np.random.uniform()
    if random_value < probability:
        # signifies up in openai gym
        return 2
    else:
         # signifies down in openai gym
        return 3

def compute_gradient(gradient_log_p, hidden_layer_values, observation_values, weights):
    """ See here: http://neuralnetworksanddeeplearning.com/chap2.html"""
    delta_L = gradient_log_p
    dC_dw2 = np.dot(hidden_layer_values.T, delta_L).ravel() #Return a contiguous flattened array.
    delta_l2 = np.outer(delta_L, weights['2'])#out[i, j] = a[i] * b[j]
    delta_l2 = relu(delta_l2)
    dC_dw1 = np.dot(delta_l2.T, observation_values)
    return {
        '1': dC_dw1,
        '2': dC_dw2
    }

def update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate):
    """ See here: http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop"""
    epsilon = 1e-5
    for layer_name in weights.keys():
        g = g_dict[layer_name]
        expectation_g_squared[layer_name] = decay_rate * expectation_g_squared[layer_name] + (1 - decay_rate) * g**2
        weights[layer_name] += (learning_rate * g)/(np.sqrt(expectation_g_squared[layer_name] + epsilon))
        g_dict[layer_name] = np.zeros_like(weights[layer_name]) # reset batch gradient buffer

def discount_rewards(rewards, gamma):
    """ Actions you took 20 steps before the end result are less important to the overall result than an action you took a step ago.
    This implements that logic by discounting the reward on previous actions based on how long ago they were taken"""
    discounted_rewards = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(xrange(0, rewards.size)):
        if rewards[t] != 0:
            running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + rewards[t]
        discounted_rewards[t] = running_add
    return discounted_rewards

def discount_with_rewards(gradient_log_p, episode_rewards, gamma):
    """ discount the gradient with the normalized rewards """
    discounted_episode_rewards = discount_rewards(episode_rewards, gamma)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_episode_rewards -= np.mean(discounted_episode_rewards)
    discounted_episode_rewards /= np.std(discounted_episode_rewards)
    return gradient_log_p * discounted_episode_rewards

In [30]:
env = gym.make("Pong-v0")
observation = env.reset()
print observation.shape
processed_observation=observation[35:195]
print processed_observation.shape

a=remove_background(processed_observation)
print a.shape
print a.type

[2017-06-01 23:46:24,435] Making new env: Pong-v0


(210, 160, 3)
(160, 160, 3)
(160, 160, 3)


AttributeError: 'numpy.ndarray' object has no attribute 'type'

In [34]:
import numpy as np
a=np.array([[[1,2,3],[1,7,8]],[[3,5,7],[1,4,5]],[[3,5,7],[1,4,5]]])
print a.shape
a[a==3] = 0
print a
b=np.array([1,2,3,4,5])
b[b==3]=0
print b
def remove_background(image):
    image[image == 4] = 0
    image[image == 3] = 0
    return image
remove_background(a)


(3, 2, 3)
[[[1 2 0]
  [1 7 8]]

 [[0 5 7]
  [1 4 5]]

 [[0 5 7]
  [1 4 5]]]
[1 2 0 4 5]


array([[[1, 2, 0],
        [1, 7, 8]],

       [[0, 5, 7],
        [1, 0, 5]],

       [[0, 5, 7],
        [1, 0, 5]]])

In [ ]:
## Architecture

# Take in inputs from the screen and preprocess them
# Pass them into an NN
# Update the weights of the NN using gradient descent
# weights['1'] - Matrix that holds weights of pixels passing into hidden layer. Dimensions: [200 x 80 x 80] -> [200 x 6400]
# weights['2'] - Matrix that holds weights of hidden layer passing into output. Dimensions: [1 x 200]

# Process is:

# processed_observations = image vector - [6400 x 1] array
# Compute hidden_layer_values = weights['1'] dot processed_observations ([200 x 6400] dot [6400 x 1]) -> [200 x 1] - this gives initial activation values.
# Next we need to transform those either via a sigmoid or an ReLU of some sort. Let's use ReLU
# ReLU(hidden_layer_values)
# Next we need to pass this one layer further
# output_layer_value = weights['2'] dot hidden_layer_values ([1 x 200] dot [200 x 1] -> [1 x 1])
# Now our output layer is the probability of going up or down. Let's make sure this output is between 0 and 1 by passing it through a sigmoid
# p = sigmoid(output_layer_value)

# Learning after round has finished:

# Figure out the result
# Compute the error
# Use the error to calculate the gradient
    # The below dimensions all assume we had exactly 10 frames in the round (not necessarily true!)
    # dC_dw2 = hidden_layer_values^T dot gradient_log_p ([1 x 2000] dot [2000 x 1] -> 1x1)
    # delta_1 = gradient_log_p outer_product weights['2'] = [2000 x 1] outer_product [1 x 200] ([2000 x 200])
    # dC_dw1 = delta_1^T dot input_observations ([200 x 2000]x dot [2000 x 64000] -> [200 x 64000])

# After some batch size of rounds has finished,
    # Use rmsprop to move weights['1'] and weights['2'] in the direction of the gradient
# Repeat!

import gym
import numpy as np
import pickle

def downsample(image):
    # Take only alternate pixels - basically halves the resolution of the image (which is fine for us)
    return image[::2, ::2, :]#read a='hello' a[::2] = 'hlo'

def remove_color(image):
    """Convert all color (RGB is the third dimension in the image)"""
    return image[:, :, 0]

def remove_background(image):
    image[image == 144] = 0 #remove the number 144,list is not run , and np.array is ok a = np.array[12,23,144] result is a = [12,23,0]
    image[image == 109] = 0
    return image

def preprocess_observations(input_observation, prev_processed_observation, input_dimensions):
    """ convert the 210x160x3 uint8 frame into a 6400 float vector """
    processed_observation = input_observation[35:195] # crop
    processed_observation = downsample(processed_observation)
    processed_observation = remove_color(processed_observation)
    processed_observation = remove_background(processed_observation)
    processed_observation[processed_observation != 0] = 1 # everything else (paddles, ball) just set to 1
    # Convert from 80 x 80 matrix to 1600 x 1 matrix
    processed_observation = processed_observation.astype(np.float).ravel()

    # subtract the previous frame from the current one so we are only processing on changes in the game
    if prev_processed_observation is not None:
        input_observation = processed_observation - prev_processed_observation
    else:
        input_observation = np.zeros(input_dimensions)
    # store the previous frame so we can subtract from it next time
    prev_processed_observations = processed_observation
    return input_observation, prev_processed_observations


def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def relu(vector):
    vector[vector < 0] = 0
    return vector

def apply_neural_nets(observation_matrix, weights):
    """ Based on the observation_matrix and weights, compute the new hidden layer values and the new output layer values"""
    hidden_layer_values = np.dot(weights['1'], observation_matrix)
    hidden_layer_values = relu(hidden_layer_values)
    output_layer_values = np.dot(hidden_layer_values, weights['2'])
    output_layer_values = sigmoid(output_layer_values)
    return hidden_layer_values, output_layer_values

def choose_action(probability):
    random_value = np.random.uniform()
    if random_value < probability:
        # signifies up in openai gym
        return 2
    else:
         # signifies down in openai gym
        return 3

def compute_gradient(gradient_log_p, hidden_layer_values, observation_values, weights):
    """ See here: http://neuralnetworksanddeeplearning.com/chap2.html"""
    delta_L = gradient_log_p
    dC_dw2 = np.dot(hidden_layer_values.T, delta_L).ravel()
    delta_l2 = np.outer(delta_L, weights['2'])
    delta_l2 = relu(delta_l2)
    dC_dw1 = np.dot(delta_l2.T, observation_values)
    return {
        '1': dC_dw1,
        '2': dC_dw2
    }

def update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate):
    """ See here: http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop"""
    epsilon = 1e-5
    for layer_name in weights.keys():
        g = g_dict[layer_name]
        expectation_g_squared[layer_name] = decay_rate * expectation_g_squared[layer_name] + (1 - decay_rate) * g**2
        weights[layer_name] += (learning_rate * g)/(np.sqrt(expectation_g_squared[layer_name] + epsilon))
        g_dict[layer_name] = np.zeros_like(weights[layer_name]) # reset batch gradient buffer
        


def discount_rewards(rewards, gamma):
    """ Actions you took 20 steps before the end result are less important to the overall result than an action you took a step ago.
    This implements that logic by discounting the reward on previous actions based on how long ago they were taken"""
    discounted_rewards = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(xrange(0, rewards.size)):
        if rewards[t] != 0:
            running_add = 0 # reset the sum, since this was a game boundary (pong specific!)
        running_add = running_add * gamma + rewards[t]
        discounted_rewards[t] = running_add
    return discounted_rewards

def discount_with_rewards(gradient_log_p, episode_rewards, gamma):
    """ discount the gradient with the normalized rewards """
    discounted_episode_rewards = discount_rewards(episode_rewards, gamma)
    # standardize the rewards to be unit normal (helps control the gradient estimator variance)
    discounted_episode_rewards -= np.mean(discounted_episode_rewards)
    discounted_episode_rewards /= np.std(discounted_episode_rewards)
    return gradient_log_p * discounted_episode_rewards


def main():
    env = gym.make("Pong-v0")
    env.mode = 'fast'
    observation = env.reset() # This gets us the image

    # hyperparameters
    episode_number = 0
    batch_size = 10
    gamma = 0.99 # discount factor for reward
    decay_rate = 0.99
    num_hidden_layer_neurons = 200
    input_dimensions = 80 * 80
    learning_rate = 1e-4

    episode_number = 0
    reward_sum = 0
    running_reward = None
    prev_processed_observations = None

    weights = {
        '1': np.random.randn(num_hidden_layer_neurons, input_dimensions) / np.sqrt(input_dimensions),
        '2': np.random.randn(num_hidden_layer_neurons) / np.sqrt(num_hidden_layer_neurons)
    }

    # To be used with rmsprop algorithm (http://sebastianruder.com/optimizing-gradient-descent/index.html#rmsprop)
    expectation_g_squared = {}
    g_dict = {}
    for layer_name in weights.keys():
        expectation_g_squared[layer_name] = np.zeros_like(weights[layer_name])
        g_dict[layer_name] = np.zeros_like(weights[layer_name])

    episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [], [], [], []

    i = 1
    while True:
        env.render()
        processed_observations, prev_processed_observations = preprocess_observations(observation, prev_processed_observations, input_dimensions)
        hidden_layer_values, up_probability = apply_neural_nets(processed_observations, weights)
    
        episode_observations.append(processed_observations)
        episode_hidden_layer_values.append(hidden_layer_values)

        action = choose_action(up_probability)

        # carry out the chosen action
        observation, reward, done, info = env.step(action)

        reward_sum += reward
        episode_rewards.append(reward)

        # see here: http://cs231n.github.io/neural-networks-2/#losses
        fake_label = 1 if action == 2 else 0
        loss_function_gradient = fake_label - up_probability
        episode_gradient_log_ps.append(loss_function_gradient)


        if done: # an episode finished
            episode_number += 1

            # Combine the following values for the episode
            episode_hidden_layer_values = np.vstack(episode_hidden_layer_values)
            episode_observations = np.vstack(episode_observations)
            episode_gradient_log_ps = np.vstack(episode_gradient_log_ps)
            episode_rewards = np.vstack(episode_rewards)

            # Tweak the gradient of the log_ps based on the discounted rewards
            episode_gradient_log_ps_discounted = discount_with_rewards(episode_gradient_log_ps, episode_rewards, gamma)

            gradient = compute_gradient(
              episode_gradient_log_ps_discounted,
              episode_hidden_layer_values,
              episode_observations,
              weights
            )

            # Sum the gradient for use when we hit the batch size
            for layer_name in gradient:
                g_dict[layer_name] += gradient[layer_name]

            if episode_number % batch_size == 0:
                update_weights(weights, expectation_g_squared, g_dict, decay_rate, learning_rate)
                
                if i==1:
                    with open('file1.txt', 'rb') as handle:
                        weights = pickle.loads(handle.read())
                        i = i + 1
                    
                    print "1111111111111111111111111"
                    print weights
                    
                else:
                    with open('file1.txt', 'wb') as handle:
                        pickle.dump(weights, handle)
                    i = i + 1
                    
                    print "22222222222222222222222222"
                    print weights


            episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [], [], [], [] # reset values
            observation = env.reset() # reset env
            running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
            print 'resetting env. episode reward total was %f. running mean: %f' % (reward_sum, running_reward)
            reward_sum = 0
            prev_processed_observations = None

main()

resetting env. episode reward total was -20.000000. running mean: -20.000000
resetting env. episode reward total was -21.000000. running mean: -20.010000
resetting env. episode reward total was -21.000000. running mean: -20.019900
resetting env. episode reward total was -21.000000. running mean: -20.029701
resetting env. episode reward total was -21.000000. running mean: -20.039404
resetting env. episode reward total was -20.000000. running mean: -20.039010
resetting env. episode reward total was -20.000000. running mean: -20.038620
resetting env. episode reward total was -21.000000. running mean: -20.048234
resetting env. episode reward total was -21.000000. running mean: -20.057751
1111111111111111111111111
{'1': array([[ 0.00987915,  0.01730566, -0.01047021, ...,  0.05431548,
         0.11642475, -0.00202735],
       [-0.01013475, -0.00040148,  0.01210236, ...,  0.04513826,
         0.02527788,  0.006018  ],
       [-0.01541359,  0.03231854,  0.00090933, ...,  0.04973625,
         0

resetting env. episode reward total was -9.000000. running mean: -19.334130
resetting env. episode reward total was -15.000000. running mean: -19.290789
resetting env. episode reward total was -16.000000. running mean: -19.257881
resetting env. episode reward total was -14.000000. running mean: -19.205302
resetting env. episode reward total was -19.000000. running mean: -19.203249
resetting env. episode reward total was -18.000000. running mean: -19.191217
resetting env. episode reward total was -14.000000. running mean: -19.139304
resetting env. episode reward total was -21.000000. running mean: -19.157911
resetting env. episode reward total was -12.000000. running mean: -19.086332
22222222222222222222222222
{'1': array([[ 0.00987915,  0.01730566, -0.01047021, ...,  0.05431548,
         0.11690057, -0.00202735],
       [-0.01013475, -0.00040148,  0.01210236, ...,  0.04506081,
         0.02478506,  0.006018  ],
       [-0.01541359,  0.03231854,  0.00090933, ...,  0.04963823,
         0

resetting env. episode reward total was -11.000000. running mean: -18.590320
resetting env. episode reward total was -14.000000. running mean: -18.544417
resetting env. episode reward total was -14.000000. running mean: -18.498973
resetting env. episode reward total was -14.000000. running mean: -18.453983
resetting env. episode reward total was -18.000000. running mean: -18.449443
resetting env. episode reward total was -19.000000. running mean: -18.454949
resetting env. episode reward total was -18.000000. running mean: -18.450399
resetting env. episode reward total was -11.000000. running mean: -18.375895
resetting env. episode reward total was -16.000000. running mean: -18.352136
22222222222222222222222222
{'1': array([[ 0.00987915,  0.01730566, -0.01047021, ...,  0.05431548,
         0.11673605, -0.00202735],
       [-0.01013475, -0.00040148,  0.01210236, ...,  0.0450006 ,
         0.02453359,  0.006018  ],
       [-0.01541359,  0.03231854,  0.00090933, ...,  0.04956226,
         

resetting env. episode reward total was 2.000000. running mean: -17.689077
resetting env. episode reward total was -12.000000. running mean: -17.632186
resetting env. episode reward total was -17.000000. running mean: -17.625865
resetting env. episode reward total was -14.000000. running mean: -17.589606
resetting env. episode reward total was -13.000000. running mean: -17.543710
resetting env. episode reward total was -13.000000. running mean: -17.498273
resetting env. episode reward total was -14.000000. running mean: -17.463290
resetting env. episode reward total was -14.000000. running mean: -17.428657
resetting env. episode reward total was -19.000000. running mean: -17.444371
22222222222222222222222222
{'1': array([[ 0.00987915,  0.01730566, -0.0104785 , ...,  0.05428426,
         0.11670245, -0.00202735],
       [-0.01013475, -0.00040148,  0.01210269, ...,  0.04499949,
         0.02392169,  0.006018  ],
       [-0.01541359,  0.03231854,  0.00090975, ...,  0.04956085,
         0.

resetting env. episode reward total was -13.000000. running mean: -16.791409
resetting env. episode reward total was -16.000000. running mean: -16.783495
resetting env. episode reward total was -13.000000. running mean: -16.745660
resetting env. episode reward total was -16.000000. running mean: -16.738203
resetting env. episode reward total was -16.000000. running mean: -16.730821
resetting env. episode reward total was -15.000000. running mean: -16.713513
resetting env. episode reward total was -18.000000. running mean: -16.726378
resetting env. episode reward total was -13.000000. running mean: -16.689114
resetting env. episode reward total was -11.000000. running mean: -16.632223
22222222222222222222222222
{'1': array([[ 0.00987915,  0.01730566, -0.0104785 , ...,  0.05428426,
         0.11652687, -0.00202735],
       [-0.01013475, -0.00040148,  0.01210269, ...,  0.04497447,
         0.02373161,  0.006018  ],
       [-0.01541359,  0.03231854,  0.00090975, ...,  0.04952917,
         

resetting env. episode reward total was -7.000000. running mean: -16.167881
resetting env. episode reward total was -15.000000. running mean: -16.156202
resetting env. episode reward total was -12.000000. running mean: -16.114640


True
